Clone MiCE repo:

In [ ]:
!git clone --branch pos_Adj https://github.com/thanoskaravangelis/mice

In [ ]:
%cd /kaggle/working/mice

MiCE's user guide

In [ ]:
!pip install -r requirements.txt

In [ ]:
!wget https://storage.googleapis.com/allennlp-public-models/mice-imdb-predictor.tar.gz
!wget https://storage.googleapis.com/allennlp-public-models/mice-imdb-editor.pth

In [ ]:
!mkdir -p trained_predictors/imdb/model
!mkdir -p results/imdb/editors/mice

In [ ]:
!mv mice-imdb-predictor.tar.gz trained_predictors/imdb/model/model.tar.gz
!mv mice-imdb-editor.pth results/imdb/editors/mice/imdb_editor.pth 

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
#@title Generate edits from MiCE with targeted POS tokens from one of the options and with given inputs path { run: "auto" }

inputs_path = "/kaggle/input/imdbdataforthesis/drive-download-20230226T193814Z-001/imdb_431"
targeted_pos_tag = "ADJ" #@param ["ADJ", "VERB", "NOUN", "PRON", "ADV"]
print('Your targeted POS tag is: ', targeted_pos_tag)

Helper functions to read csv for new round of edits

In [ ]:
import numpy as np
def read_edits(path):
    edits = pd.read_csv(path, sep="\t", lineterminator="\n", error_bad_lines=False, warn_bad_lines=True)

    if edits['new_pred'].dtype == pd.np.dtype('float64'):
        edits['new_pred'] = edits.apply(lambda row: str(int(row['new_pred']) if not np.isnan(row['new_pred']) else ""), axis=1)
        edits['orig_pred'] = edits.apply(lambda row: str(int(row['orig_pred']) if not np.isnan(row['orig_pred']) else ""), axis=1)
        edits['contrast_pred'] = edits.apply(lambda row: str(int(row['contrast_pred']) if not np.isnan(row['contrast_pred']) else ""), axis=1)
    else:
        edits['new_pred'].fillna(value="", inplace=True)
        edits['orig_pred'].fillna(value="", inplace=True)
        edits['contrast_pred'].fillna(value="", inplace=True)
    return edits

In [ ]:
def get_best_edits(edits):
    """ MiCE writes all edits that are found in Stage 2, 
    but we only want to evaluate the smallest per input. 
    Calling get_sorted_e() """
    return edits[edits['sorted_idx'] == 0]

In [ ]:
def create_files(edits, folder_name):
  parent_dir = f'./results/imdb/edits/{folder_name}'

  os.mkdir(parent_dir)

  path_pos = os.path.join(parent_dir, 'pos')
  path_neg = os.path.join(parent_dir, 'neg')
  os.mkdir(path_pos)
  os.mkdir(path_neg)

  for _, row in edits.iterrows():
    prob = row['new_pred']
    if prob == '1':
      f = open(path_pos+f"/{row['data_idx']}.txt", "x")
      f.write(row["edited_editable_seg"])
      f.close()
    elif prob == '0':
      f = open(path_neg+f"/{row['data_idx']}.txt", "x")
      f.write(row["edited_editable_seg"])
      f.close()

First round:

In [ ]:
!python run_stage_two.py -task imdb -stage2_exp mice_imdb_{targeted_pos_tag}_0 -editor_path results/imdb/editors/mice/imdb_editor.pth -targeted_pos_tag {targeted_pos_tag} -inputs_path {inputs_path}

Another 9 rounds:

In [ ]:
from tqdm import tqdm 
import pandas as pd
import os
for num_of_phase in tqdm(range (1, 10)):
    # διαβάζω ένα αρχείο csv με τα αποτελέσματα του ΠΡΟΗΓΟΥΜΕΝΟΥ ΓΥΡΟΥ 
    edits = read_edits(f"./results/imdb/edits/mice_imdb_{targeted_pos_tag}_{num_of_phase - 1}/edits.csv")
    edits = get_best_edits(edits)
    # ορίζω σαν νέο foldername την νέα φάση
    folder_name = f"mice_imdb_{targeted_pos_tag}_{num_of_phase}"
    # και φτιάχνω τα αντίστοιχα txt αρχεία
    create_files(edits, folder_name)

    os.system(f"python run_stage_two.py -task imdb -stage2_exp {folder_name} -editor_path results/imdb/editors/mice/imdb_editor.pth -targeted_pos_tag {targeted_pos_tag} -inputs_path /kaggle/working/mice/results/imdb/edits/{folder_name}")